In [1]:
import matplotlib.pyplot as plt
import os
import random
import math
# external imports
import transformers
from transformers import AutoImageProcessor, AutoModelForDepthEstimation
import torch
import torchvision
import time 
import numpy as np
from PIL import Image, ImageDraw, ImageFont  # Import ImageDraw and ImageFont
import requests
import datasets
from datasets import load_dataset
from torchvision import datasets, transforms
from tqdm import tqdm
import cv2
import pandas as pd

# Intial Tests with DepthAnything


In [ ]:
print(torch.cuda.is_available())

In [ ]:
torch.cuda.empty_cache()

# Loading and Investigating FLSea Dataset

## Dataloader

In [2]:
def shift_labels(predicted_labels, ground_truth_labels):
    # Calculate mean or median values
    predicted_mean = np.mean(predicted_labels)
    ground_truth_mean = np.mean(ground_truth_labels)

    # Calculate the shift
    shift = ground_truth_mean - predicted_mean

    # Adjust the predicted labels
    shifted_predicted_labels = predicted_labels + shift

    return shifted_predicted_labels


def plot_histograms(groundtruth, predicted):
    plt.figure(figsize=(10, 5))
    plt.hist(groundtruth, bins=50, alpha=0.5, color='blue', label='ground truth')
    plt.hist(predicted, bins=50, alpha=0.5, color='red', label='predicted')
    plt.title('Histogram of Depth Variables')
    plt.legend()
    plt.show()

def min_max_normalize(image):
    # Get the minimum and maximum pixel values
    min_val = np.min(image)
    max_val = np.max(image)
    if min_val == max_val:
        return np.empty((0,))
    # Normalize the image
    normalized_image = (image - min_val) / (max_val - min_val)
    return normalized_image

In [3]:
def gamma_correction(image, gamma):
    """
    Apply gamma correction to the input jpg image.
    """

    # Convert PIL image to numpy array
    image_array = np.array(image)

    # Apply gamma correction
    corrected_image_array = np.uint8(255 * (image_array / 255) ** gamma)
    
    # Ensure no negative values
    min_value = np.min(corrected_image_array)
    corrected_image_array_shifted = corrected_image_array - min_value
        
    # Convert back to PIL image
    # corrected_image = Image.fromarray(corrected_image_array_shifted)
    
    return corrected_image_array_shifted


class GammaCorrectionAndNormalization(torch.nn.Module):
    def __init__(self, gamma, stretch_factor):
        super(GammaCorrectionAndNormalization, self).__init__()
        self.gamma = gamma
        self.stretch_factor = stretch_factor

    def forward(self, img):
        # Apply gamma correction
        gamma_corrected_img = gamma_correction(img, gamma=self.gamma)

        # Convert gamma-corrected image to numpy array
        # img_array = np.array(gamma_corrected_img)

#         # Apply contrast stretching to each channel independently
#         stretched_img_array = np.zeros_like(gamma_corrected_img)
#         for i in range(gamma_corrected_img.shape[2]):  # Iterate over channels
#             channel_min = np.min(gamma_corrected_img[:, :, i])
#             channel_max = np.max(gamma_corrected_img[:, :, i])
#             stretched_img_array[:, :, i] = (gamma_corrected_img[:, :, i] - channel_min) * self.stretch_factor / (channel_max - channel_min)

#         # Calculate mean and standard deviation
#         mean_values = np.mean(gamma_corrected_img, axis=(0, 1))
#         std_values = np.std(gamma_corrected_img, axis=(0, 1))
        
        # Normalize the image using calculated mean and standard deviation
        # normalized_img_array = (gamma_corrected_img - mean_values) / std_values
        # Convert numpy array back to PIL image
        # normalized_img = Image.fromarray((normalized_img_array).astype(np.uint8))

        # Min-max normalization
        min_value = np.min(gamma_corrected_img)
        max_value = np.max(gamma_corrected_img)
        # print(min_value, max_value)
        normalized_img_array = (gamma_corrected_img - min_value) / (max_value - min_value)


        return normalized_img_array
    
    
class MinMaxScaler:
    def __init__(self, min_val, max_val):
        self.min_val = torch.tensor(min_val, dtype=torch.float32)
        self.max_val = torch.tensor(max_val, dtype=torch.float32)

    def __call__(self, tensor):
        """
        Perform Min-Max scaling on the input tensor.

        Parameters:
            tensor (Tensor): Input tensor to be scaled.

        Returns:
            Tensor: Scaled tensor in the range [0, 1].
        """
        # Perform Min-Max scaling

        # scaled_tensor = (tensor - self.min_val) / (self.max_val - self.min_val)
        
        # Scale each channel independently
        scaled_tensor = tensor.clone()  # Create a copy of the input tensor
        for i in range(3):  # Assuming RGB channels
            scaled_tensor[i] = (tensor[i] - self.min_val[i]) / (self.max_val[i] - self.min_val[i])
            
        # Clip values to ensure they are within [0, 1] range
        scaled_tensor = torch.clamp(scaled_tensor, 0, 1)

        return scaled_tensor

In [4]:
class CustomDepthDataset(torch.utils.data.Dataset):
    def __init__(self, image_dir, depth_dir, transform_img=None, transform_depth=None):
        self.image_dir = image_dir
        self.depth_dir = depth_dir
        self.transform_img = transform_img
        self.transform_depth = transform_depth

        self.image_filenames = os.listdir(image_dir)
        self.depth_filenames = os.listdir(depth_dir)

    def __len__(self):
        return len(self.image_filenames)

    def mask_edges(self, image, threshold1=50, threshold2=110, dilation_kernel_size=3):
        # Convert the image to grayscale
        gray = image.convert('L')

        # Convert grayscale image to numpy array
        gray_np = np.array(gray)

        # Apply Canny edge detection
        edges = cv2.Canny(gray_np, threshold1=threshold1, threshold2=threshold2)

        # Create a dilation kernel
        kernel = np.ones((dilation_kernel_size, dilation_kernel_size), np.uint8)

        # Dilate the edges to create a thicker boundary
        dilated_edges = cv2.dilate(edges, kernel)

        # Create a mask for pixels near edges
        mask = dilated_edges != 0

        # Apply the mask to the depth image
        depth_array = np.array(image)
        depth_array[mask] = 0

        return Image.fromarray(depth_array)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.image_filenames[idx])
        depth_name = os.path.join(self.depth_dir, self.depth_filenames[idx])

        image = Image.open(img_name)
        depth = Image.open(depth_name)

        # Apply edge mask to the depth image
        depth = self.mask_edges(depth)

        if self.transform_img:
            image = self.transform_img(image)
        if self.transform_depth:
            depth = self.transform_depth(depth)

        return image, depth


In [5]:
image_path = 'C:\\Users\\susan\\Documents\\University\\MIR\\DataDriven\\FLSea\\canyons\\tiny_canyon\\tiny_canyon\\imgs\\'
depth_path = 'C:\\Users\\susan\\Documents\\University\\MIR\\DataDriven\\FLSea\\canyons\\tiny_canyon\\tiny_canyon\\depth\\'


# Define gamma value
gamma = 0.7
stretch_factor = 0.5
# mean_values_norm = [0.485, 0.456, 0.406]
# std_values_norm =  [0.229, 0.224, 0.225]

mean_values_norm = [0.700, 0.600, 0.200]
std_values_norm =  [0.5, 0.224, 0.1]

min_val = [-1.4000, -2.6786, -1.8800]
max_val = [0.6000, 1.7857, 8.0000]
# Create the custom transform
custom_transform = GammaCorrectionAndNormalization(gamma, stretch_factor)
scaler = MinMaxScaler(min_val, max_val)


transform_img = transforms.Compose([
    custom_transform,
    # transforms.Lambda(lambda x: gamma_correction(x, gamma=0.7)), # Apply gamma correction
    transforms.ToTensor(),  # Convert the image to a tensor 
    transforms.Normalize(mean_values_norm, std_values_norm),  # Normalize the tensor
    scaler, # Min-Max scaling  
])


transform_depth = transforms.Compose([
    transforms.ToTensor(),  # Convert the image to a tensor
])

# Create Dataset
train_data = CustomDepthDataset(image_path, depth_path, transform_img=transform_img, transform_depth=transform_depth)

# Create data loaders
batch_size = 4
dataloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=False)

In [ ]:
def Histogramize_dataloader(dataloader):
    # Initialize empty lists to store pixel values for each channel
    pixel_values_R = []
    pixel_values_G = []
    pixel_values_B = []

    # Iterate over the dataset using the DataLoader
    for images, depths in tqdm(dataloader, desc="Getting RGB values...", unit="batch") :


        # Iterate over each image in the batch
        for image in images:
            # Split the image into channels
            R, G, B = image[0].cpu().numpy(), image[1].cpu().numpy(), image[2].cpu().numpy()
    

    # Plot histograms for each channel
    fig, axs = plt.subplots(3, 1, figsize=(8, 10))

    # Plot histogram for the R channel
    axs[0].hist(np.ravel(R),  color='red')
    axs[0].set_xlabel('Pixel Value')
    axs[0].set_ylabel('Frequency')
    axs[0].set_title('Histogram of Red Channel')

    # Plot histogram for the G channel
    axs[1].hist(np.ravel(G),  color='green')
    axs[1].set_xlabel('Pixel Value')
    axs[1].set_ylabel('Frequency')
    axs[1].set_title('Histogram of Green Channel')

    # Plot histogram for the B channel
    axs[2].hist(np.ravel(B), color='blue')
    axs[2].set_xlabel('Pixel Value')
    axs[2].set_ylabel('Frequency')
    axs[2].set_title('Histogram of Blue Channel')

    plt.tight_layout()
    plt.show()

In [ ]:
Histogramize_dataloader(dataloader)

In [ ]:
# Calculate min and max values
min_values = torch.ones(3)
max_values = torch.zeros(3)
total_samples = 0

global_min = torch.ones(3)
global_max = torch.zeros(3)

for batch_imgs, _ in tqdm(dataloader, desc="Computing min max", unit="batch"):
    
    # Calculate the minimum value for each channel
    batch_min = torch.min(batch_imgs, dim=2, keepdim=True)[0]  # Take the min across height dimension
    
    batch_min = torch.min(batch_min, dim=3)[0]  # Take the min across width dimension
    
    batch_max = torch.max(batch_imgs, dim=2, keepdim=True)[0]  # Take the max across height dimension
    batch_max = torch.max(batch_max, dim=3)[0]  # Take the max across width dimension

    # find min and max in the current batch
    min_values = torch.min(batch_min, dim=0)[0].squeeze()
    max_values = torch.max(batch_max, dim=0)[0].squeeze()

    # update the global min/max them if necessary
    global_min =  torch.where(min_values < global_min, min_values, global_min)
    global_max =  torch.where(max_values > global_max, max_values, global_max)



print("Min values:", global_min)
print("Max values:", global_max)

In [ ]:
# Calculate mean and standard deviation
mean_values = torch.zeros(3)
std_values = torch.zeros(3)
total_samples = 0


for batch_imgs, _ in tqdm(dataloader, desc="Computing mean and standard deviation", unit="batch"):
    # Calculate mean and std per batch
    batch_mean = torch.mean(batch_imgs, dim=(0, 2, 3))
    batch_std = torch.std(batch_imgs, dim=(0, 2, 3))
    
    # Update total mean and std
    mean_values += batch_mean
    std_values += batch_std
    
    # Update total number of samples
    total_samples += batch_imgs.size(0)

# Calculate final mean and std
mean_values /= total_samples
std_values /= total_samples

print("Mean values:", mean_values)
print("Standard deviation values:", std_values)

### Sample depths from dataset

In [6]:
def sample_depth(dataloader, k, model, image_processor):
    images_container = []
    depths_container = []
    
    i = 0
    
    # Create a grid to display the images
    fig, axes = plt.subplots(k, 3, figsize=(10, 2*k))

    # Iterate over batches in the DataLoader
    for images, depths in dataloader:
        
        i +=1
        
        # Stop if processed k batches
        if i >= k:
            break
  
        # Prepare images for the model
        inputs = image_processor(images=images, return_tensors="pt")
        
        with torch.no_grad():
            # Forward pass through the model
            outputs = model(**inputs)
            predicted_depths = outputs.predicted_depth

        # Iterate over images in the batch
        for j in range(images.shape[0]):
            # Interpolate to original size
            prediction = torch.nn.functional.interpolate(
                predicted_depths[j].unsqueeze(0).unsqueeze(0),
                size=[images.shape[2], images.shape[3]],
                mode="bicubic",
                align_corners=False,
            )

            # Convert depth prediction to numpy array
            depth_output = prediction.squeeze().cpu().numpy()

            # Plot original image
            axes[j, 0].imshow(images[j].permute(1, 2, 0))
            axes[j, 0].axis('off')
            axes[j, 0].set_title(f'Image {i*k + j}')

            # Plot depth output
            axes[j, 1].imshow(depth_output, cmap='plasma')
            axes[j, 1].axis('off')
            axes[j, 1].set_title(f'Depth {i*k + j}')

            # Plot ground truth depth
            axes[j, 2].imshow(depths[j].squeeze(), cmap='plasma_r')
            axes[j, 2].axis('off')
            axes[j, 2].set_title(f'Ground Truth Depth {i*k + j}')


            images_container.append(images[j].permute(1, 2, 0).numpy())
            depths_container.append(depths[j].numpy())

    plt.show()

    return np.asarray(images_container), np.asarray(depths_container)


# Baseline Results -> No training

We can implement some basic metrics for getting baseline results for the entire dataset. DepthAnything used these metrics.

In [7]:
def absolute_relative_error(y_pred, y_true):
    """
    Calculate the Mean Absolute Relative Error (MARE).

    Parameters:
    y_pred : torch.Tensor
        Predicted depth values.
    y_true : torch.Tensor
        Ground truth depth values.

    Returns:
    float
        Mean Absolute Relative Error (MARE).
    """
    # Mask out both zeros and NaNs
    mask = (y_true != 0) & (~np.isnan(y_true))
    
    # Apply the mask to both predicted and true values
    y_pred_masked = y_pred[mask]
    y_true_masked = y_true[mask]

    # Calculate absolute relative error
    absolute_relative_error = np.abs(y_pred_masked - y_true_masked) / y_true_masked
    
    # Calculate mean excluding masked values
    mare = np.mean(absolute_relative_error)
    
    return mare.item() 

In [8]:
def root_mean_squared_error(y_pred, y_true): 
    mse = np.mean((y_pred - y_true)**2)
    rmse = np.sqrt(mse)
    return rmse

In [9]:
def delta1_metric(y_pred, y_true, threshold=1.25):
    """
    Calculate the δ1 metric for monocular depth estimation.

    Parameters:
    y_pred : torch.Tensor
        Predicted depth values.
    y_true : torch.Tensor
        Ground truth depth values.
    threshold : float, optional
        Threshold for considering a pixel as correctly estimated (default is 1.25).

    Returns:
    float
        Percentage of pixels for which max(d*/d, d/d*) < threshold.
    """

     # Mask out both zeros and NaNs
    # nan_mask = np.isnan(y_true) | np.isnan(y_pred) | (y_true == 0)
    
    # Apply the mask to both predicted and true values
    # y_pred_masked = y_pred[~nan_mask]
    # y_true_masked = y_true[~nan_mask]
    
    # Calculate element-wise ratios
    ratio_1 = y_true / (y_pred + 1e-7)  # Adding epsilon to avoid division by zero
    ratio_2 = y_pred / (y_true + 1e-7) # Adding epsilon to avoid division by zero
    
    # Calculate element-wise maximum ratio
    max_ratio = np.maximum(ratio_1, ratio_2)

    # print(max_ratio)
    # Count the number of pixels where max_ratio < threshold
    num_correct_pixels = np.sum(max_ratio < threshold)
    # print("num correct pixels", num_correct_pixels)
    
    # Calculate the percentage of pixels satisfying the condition
    total_pixels = len(y_true)
    # print("num total_pixels",total_pixels)
    ratio_correct = num_correct_pixels / total_pixels
    return ratio_correct
    

In [10]:
def mask_edges(image, threshold1=50, threshold2=110, dilation_kernel_size=10):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Canny edge detection
    edges = cv2.Canny(gray, threshold1=threshold1, threshold2=threshold2)

    # Create a dilation kernel
    kernel = np.ones((dilation_kernel_size, dilation_kernel_size), np.uint8)

    # Dilate the edges to create a thicker boundary
    dilated_edges = cv2.dilate(edges, kernel)

    # Invert the dilated edges mask
    dilated_edges = cv2.bitwise_not(dilated_edges)

    # Create a black background image
    background = np.zeros_like(image)

    # Copy the original image where the edges are not detected
    masked_image = cv2.bitwise_and(image, image, mask=dilated_edges)

    return masked_image


Let us now get the depths for the whole dataset, and compute baseline results for MAE and delta1. 

In [25]:
#https://huggingface.co/docs/transformers/main/en/preprocessing#computer-vision
def evaluate_model(model, dataloader, device):
    model.eval()
    mae = 0
    delta1 = 0
    total = 0
    rmse = 0
    num_batch = 0
    skipped = 0

    # since we're not training, we don't need to calculate the gradients for our outputs
    for data, labels in tqdm(dataloader, desc="Computing metrics ", unit="batch"):
        inputs = image_processor(images=data, return_tensors="pt", do_rescale= False).to(device)

        with torch.no_grad():
            outputs = model(**inputs)
            predicted_depth = outputs.predicted_depth
            # interpolate to original size
            prediction = torch.nn.functional.interpolate(
                predicted_depth.unsqueeze(1),
                size=[data.shape[2], data.shape[3]],
                mode="bicubic",
                align_corners=False,
            )

            # calculate prediction
            output = prediction.squeeze().cpu().numpy()
            if output.size == 0:
                print("Skipping image")
                skipped += 1
                continue
            else:
                normalized_out = min_max_normalize(output)
                if normalized_out.size == 0:
                    print("Skipping image")
                    skipped += 1
                    continue
            
            labels_normalized = min_max_normalize(labels.squeeze().numpy())
            if labels_normalized.size == 0:
                print("Skipping image")
                skipped += 1
                continue
            
           
            # Create a mask for non-zero values in labels
            mask = labels_normalized != 0
            # Apply the mask to both predictions and labels
            normalized_out_masked = normalized_out[mask]
            labels_normalized_masked = labels_normalized[mask]
            
             # Create a mask for non-zero values in predictions
            mask2 = normalized_out_masked != 0
            # Apply the mask to both predictions and labels
            normalized_out_masked2 = normalized_out_masked[mask2]
            labels_normalized_masked2 = labels_normalized_masked[mask2]
            
            if labels_normalized_masked.size == 0:
                print("Skipping image")
                skipped += 1
                continue
                                    
#             final_outputs = normalized_out_masked
#             final_labels = labels_normalized_masked

            shifted_predicted_labels = shift_labels(normalized_out_masked, labels_normalized_masked)
            # mask_outliers = shifted_predicted_labels <= 1
            
            final_outputs = shifted_predicted_labels #[mask_outliers]
            final_labels = labels_normalized_masked #[mask_outliers]
            
            # Plot histograms after shifting
            # plot_histograms(final_labels, final_outputs)

            # print("shape masked labels", labels_normalized_masked.shape)
            # print(labels_normalized_masked)
            
            single_mae = absolute_relative_error(final_outputs, final_labels)
            if math.isnan(single_mae):
                print("Skipping image")
                skipped += 1
                continue
            mae += single_mae

            rmse += root_mean_squared_error(final_outputs, final_labels)
            # print("rmse", root_mean_squared_error(normalized_out_masked, labels_normalized_masked))
            
            delta1 += delta1_metric(final_outputs, final_labels, 1.25)
            # print("delta1", delta1_metric(normalized_out_masked, labels_normalized_masked))
            num_batch += 1
            # print("MAE ", mae/num_batch, ", RMSE ", rmse/num_batch, ", delta ", delta1/num_batch)


    print("total", num_batch)
    print("mae", mae)
    print("delta1", delta1)
    print("rmse", rmse)
    total_mae = mae/num_batch
    total_delta = delta1/num_batch
    total_rmse = rmse/num_batch
    

    return total_mae, total_delta, total_rmse

In [26]:
image_path = 'C:\\Users\\susan\\Documents\\University\\MIR\\DataDriven\\FLSea\\canyons\\tiny_canyon\\tiny_canyon\\imgs\\'
depth_path = 'C:\\Users\\susan\\Documents\\University\\MIR\\DataDriven\\FLSea\\canyons\\tiny_canyon\\tiny_canyon\\depth\\'


# Define gamma value
gamma = 0.7
stretch_factor = 0.5

mean_values_norm = [0.700, 0.600, 0.200]
std_values_norm = [0.5, 0.224, 0.1]

min_val = [-1.4000, -2.6786, -1.8800]
max_val = [0.6000, 1.7857, 8.0000]
# Create the custom transform
custom_transform = GammaCorrectionAndNormalization(gamma, stretch_factor)
scaler = MinMaxScaler(min_val, max_val)


transform_img = transforms.Compose([
    custom_transform,
    # transforms.Lambda(lambda x: gamma_correction(x, gamma=0.7)), # Apply gamma correction
    transforms.ToTensor(),  # Convert the image to a tensor 
    transforms.Normalize(mean_values_norm, std_values_norm),  # Normalize the tensor
    scaler, # Min-Max scaling  
])


transform_depth = transforms.Compose([
    transforms.ToTensor(),  # Convert the image to a tensor
])

# Create Dataset
train_data = CustomDepthDataset(image_path, depth_path, transform_img=transform_img, transform_depth=transform_depth)

# Create data loaders
batch_size = 4
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=False)

In [27]:
image_processor = AutoImageProcessor.from_pretrained("LiheYoung/depth-anything-small-hf")
model = AutoModelForDepthEstimation.from_pretrained("LiheYoung/depth-anything-small-hf")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

result = evaluate_model(model, train_loader, device)
result

Computing metrics :  94%|█████████████████████████████████████████████████████▊   | 239/253 [11:16<00:35,  2.53s/batch]

Skipping image


Computing metrics :  95%|██████████████████████████████████████████████████████   | 240/253 [11:19<00:32,  2.48s/batch]

Skipping image


Computing metrics :  95%|██████████████████████████████████████████████████████▎  | 241/253 [11:21<00:29,  2.49s/batch]

...

Computing metrics :  96%|██████████████████████████████████████████████████████▌  | 242/253 [11:24<00:27,  2.52s/batch]

Skipping image


Computing metrics : 100%|█████████████████████████████████████████████████████████| 253/253 [11:54<00:00,  2.82s/batch]

total 249
mae 180.81666080653667
delta1 60.8162394238533
rmse 86.1224811822176


(0.726171328540308, 0.24424192539700118, 0.3458734184024803)

In [ ]:
image_processor = AutoImageProcessor.from_pretrained("LiheYoung/depth-anything-large-hf")
model = AutoModelForDepthEstimation.from_pretrained("LiheYoung/depth-anything-large-hf")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

result_large = evaluate_model(model, train_loader, device)
result_large

In [29]:
results_nopre = (0.6780142457071079, 0.25126540302579914, 0.33210749553149965)

In [30]:
results_withpre = (0.726171328540308, 0.24424192539700118, 0.3458734184024803)

In [31]:
# import pandas to use pandas DataFrame
log10_mae_nopre = np.log10(results_nopre[0])
log10_rmse_nopre = np.log10(results_nopre[2])

log10_mae_withpre = np.log10(results_withpre[0])
log10_rmse_withpre = np.log10(results_withpre[2])

nopre = ("DepthAnything-Small: No edge-detection", results_nopre[1], results_nopre[0], log10_mae_nopre, results_nopre[2], log10_rmse_nopre)
withpre = ("DepthAnything-Small: With edge-detection", results_withpre[1], results_withpre[0], log10_mae_withpre, results_withpre[2], log10_rmse_withpre)

# data in the form of list of tuples
data = [nopre, withpre]
 
# create DataFrame using data
df = pd.DataFrame(data, columns =['Model', 'Delta1', 'AbsRel', 'AbsRel Log', 'RMSE', 'RMSE Log'])
df

,Model,Delta1,AbsRel,AbsRel Log,RMSE,RMSE Log
0,DepthAnything-Small: No edge-detection,0.251265,0.678014,-0.168761,0.332107,-0.478721
1,DepthAnything-Small: With edge-detection,0.244242,0.726171,-0.138961,0.345873,-0.461083


In [ ]:
image_path_enhanced = "C:\\Users\\susan\\Documents\\University\\MIR\\DataDriven\\FLSea\\canyons\\tiny_canyon\\tiny_canyon\\seaErra"
# Load the data
train_data_enhanced = SegmentationDataset(image_path_enhanced, depth_path, transforms=data_transforms)
f
device = 'cuda' if torch.cuda.is_available() else 'cpu' 
model.to(device)


# Create data loaders
batch_size = 16
train_loader = torch.utils.data.DataLoader(train_data_enhanced, batch_size=batch_size, shuffle=True)

result = evaluate_model(model, train_loader, device)
result